# Projet Final Apache Spark

**Nom Etudiant :**  **THIOUBE**

**Prenom Etudiant:**  **El Hadji Mamadou**

**Classe :**  **Promo 2 Intelligence Artificielle**


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Samedi 31 Juillet 2021  à 23h 59** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [1]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.6:4041
SparkContext available as 'sc' (version = 3.2.0, master = local[*], app id = local-1653748751452)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [2]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [3]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [4]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [5]:
fireDF.cache()

res0: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [6]:
fireDF.count()

res1: Long = 175296


In [8]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [9]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [10]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [11]:
// Reponse 1

val NtypeAppelDistinct = fireDF.select("CallType").distinct().count()

NtypeAppelDistinct: Long = 30


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [12]:
// Reponse 2

val TypeApDiff = fireDF.select("CallType").where(col("CallType") =!= "").distinct()
TypeApDiff.show(false)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Marine Fire                                 |
|Aircraft Emergency                          |
|Confined Space / Structure Collapse         |
|Administrative                              |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Watercraft in Distress                      |
|Explosion                                   |
|Oil Spill                                   |
|Vehicle Fire                                |
|Suspicious Package                          |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Assist Polic

TypeApDiff: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string]


### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [13]:
// Reponse 3
/*colonne Delay renommée ReponseDelayedinMins
Nouveau dataframe
*/
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")

val TypeA = newFireDF
            .select("ResponseDelayedinMins", "CallType", "IncidentNumber")
            .filter($"ResponseDelayedinMins">5)
            .show()

+---------------------+--------------------+--------------+
|ResponseDelayedinMins|            CallType|IncidentNumber|
+---------------------+--------------------+--------------+
|                 5.35|    Medical Incident|       2003409|
|                 6.25|    Medical Incident|       2003642|
|                  5.2|    Medical Incident|       2003818|
|                  5.6|Citizen Assist / ...|       2004152|
|                 7.25|    Medical Incident|       2004216|
|            11.916667|    Medical Incident|       2004408|
|             5.116667|    Medical Incident|       2004521|
|             8.633333|    Medical Incident|       2004528|
|             95.28333|    Medical Incident|       2004528|
|                 5.45|    Medical Incident|       2004610|
|                  7.6|    Medical Incident|       2004641|
|             6.133333|    Medical Incident|       2004721|
|            5.1833334|      Structure Fire|       2004724|
|            6.9166665|    Medical Incid

newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
TypeA: Unit = ()


### Transformations des dates  
Maintenant nous allons d'abord:  
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard    
2. Retourner le Dataframe transformée  
3. Mettre en cache le nouveau DataFrame  

In [14]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res7: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

In [15]:
//Reponse 4
/*
Ecrire ici votre code
*/
val applusFreq = fireDF.groupBy("CallType")
                .agg(count("*") as "Nbre d'appels")
                .orderBy(desc("Nbre d'appels"))

applusFreq.show()

+--------------------+-------------+
|            CallType|Nbre d'appels|
+--------------------+-------------+
|    Medical Incident|       113794|
|      Structure Fire|        23319|
|              Alarms|        19406|
|   Traffic Collision|         7013|
|Citizen Assist / ...|         2524|
|               Other|         2166|
|        Outside Fire|         2094|
|        Vehicle Fire|          854|
|Gas Leak (Natural...|          764|
|        Water Rescue|          755|
|Odor (Strange / U...|          490|
|   Electrical Hazard|          482|
|Elevator / Escala...|          453|
|Smoke Investigati...|          391|
|          Fuel Spill|          193|
|              HazMat|          124|
|Industrial Accidents|           94|
|           Explosion|           89|
|Train / Rail Inci...|           57|
|  Aircraft Emergency|           36|
+--------------------+-------------+
only showing top 20 rows



applusFreq: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string, Nbre d'appels: bigint]


### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [24]:
//Reponse 5-a
/*
Ecrire ici votre code
*/
val BoitPostFreq = fireDF.groupBy("Box").agg(count("*") as "Total").orderBy(desc("Total")) 
BoitPostFreq.show()

+----+-----+
| Box|Total|
+----+-----+
|2251| 1911|
|1453| 1790|
|1461| 1552|
|5236| 1269|
|2252| 1246|
|1545| 1231|
|2248| 1122|
|1456| 1090|
|1554| 1052|
|1365| 1031|
|1455|  966|
|1546|  949|
|2242|  896|
|2318|  880|
|3121|  870|
|2931|  734|
|2212|  651|
|1366|  647|
|1555|  644|
|1364|  636|
+----+-----+
only showing top 20 rows



BoitPostFreq: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Box: string, Total: bigint]


### Question 5-a
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [17]:
//Reponse 5-b
/*
Ecrire ici votre code
*/

val fewFireDF = fireDF.select("Neighborhood", "ZipCode")
                .filter($"ZipCode" === 94102 or $"ZipCode" === 94103)
                .distinct
fewFireDF.show()

+--------------------+-------+
|        Neighborhood|ZipCode|
+--------------------+-------+
|        Potrero Hill|  94103|
|    Western Addition|  94102|
|          Tenderloin|  94102|
|            Nob Hill|  94102|
| Castro/Upper Market|  94103|
|     South of Market|  94102|
|     South of Market|  94103|
|        Hayes Valley|  94103|
|Financial Distric...|  94102|
|         Mission Bay|  94103|
|          Tenderloin|  94103|
|Financial Distric...|  94103|
|        Hayes Valley|  94102|
|             Mission|  94103|
+--------------------+-------+



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Neighborhood: string, ZipCode: int]


### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [21]:
//Reponse 6
/*
Ecrire ici votre code
*/
fireDF.select("Delay")
        .describe()
        .show()

+-------+-----------------+
|summary|            Delay|
+-------+-----------------+
|  count|           175296|
|   mean|3.892364154521585|
| stddev|9.378286226254197|
|    min|      0.016666668|
|    max|          1844.55|
+-------+-----------------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [18]:
//Reponse 7-a
/*
Ecrire ici votre code
*/

val = nbr_annéedistinct

### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [23]:
//Reponse 7-b
/*
Ecrire ici votre code
*/

val SemAn2018 = fireTSDF.filter(year($"IncidentDate")===2018 && $"CallFinalDisposition"==="Fire")
                .select(max(weekofyear($"IncidentDate")) as "La semaine avec plus d'incendie 2018")
SemAn2018.show()

+------------------------------------+
|La semaine avec plus d'incendie 2018|
+------------------------------------+
|                                  45|
+------------------------------------+



SemAn2018: org.apache.spark.sql.DataFrame = [La semaine avec plus d'incendie 2018: int]


### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [20]:
//Reponse 8
/*
Ecrire ici votre code
*/


### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [21]:
//Reponse 9
/*
Ecrire ici votre code
*/

### Question 10
**Comment relire les données stockée en format Parquet?**

In [22]:
//Reponse 10
/*
Ecrire ici votre code
*/

## FIN